In [ ]:
### DB AND TABLE SETUP
import sqlite3, io, pandas as pd, base64, matplotlib.pyplot as plt, sys, os, glob, pytz, IPython.core.display as ip, plotly.express as px
from IPython.display import display, HTML
from datetime import datetime

surveyYear = '2023'
aboveDamOnly = True

additionalFilterForAboveDam = "AND CAST(Distance AS int) > 310" if aboveDamOnly else ""

surveyURIs = {'2019':'https://five.epicollect.net/api/export/entries/salmon-survey-2019?form_ref=397fba6ecc674b74836efc190840c42d_5d6f454667a28&per_page=1000',
              '2020':'https://five.epicollect.net/api/export/entries/salmon-survey-2020?form_ref=f550ab6c4dab44f49bcc33b7c1904be9_5d6f454667a28&per_page=1000',
              '2021':'https://five.epicollect.net/api/export/entries/salmon-survey-2021?form_ref=ad5ffedf0a3246a18934e6ec36ed9569_5d6f454667a28&per_page=1000',
              '2022':'https://five.epicollect.net/api/export/entries/salmon-survey-2022?form_ref=d46b5d8451f8410ea407bae5c8eb9f49_5d6f454667a28&per_page=1000'}
salmonURIs = {'2019':'https://five.epicollect.net/api/export/entries/salmon-survey-2019?form_ref=397fba6ecc674b74836efc190840c42d_5d6f509867795&per_page=1000',
              '2020':'https://five.epicollect.net/api/export/entries/salmon-survey-2020?form_ref=f550ab6c4dab44f49bcc33b7c1904be9_5d6f509867795&per_page=1000',
              '2021':'https://five.epicollect.net/api/export/entries/salmon-survey-2021?form_ref=ad5ffedf0a3246a18934e6ec36ed9569_5d6f509867795&per_page=1000',
              '2022':'https://five.epicollect.net/api/export/entries/salmon-survey-2022?form_ref=d46b5d8451f8410ea407bae5c8eb9f49_5d6f509867795&per_page=1000',
              '2023':'https://kf.kobotoolbox.org/api/v2/assets/a6dEG7tnrtwjrmituAdL5k/data/?format=json'}

IN_COLAB = 'google.colab' in sys.modules

def getFigureAsHTML():
    IObytes = io.BytesIO()
    plt.savefig(IObytes, format = 'png')
    IObytes.seek(0)
    encodedPlot = base64.b64encode(IObytes.read()).decode("utf-8")
    return '<img src=\'data:image/png;base64,{}\'>'.format(encodedPlot)

#for running locally
def clearPreviousReports():
    for fileName in glob.glob('*salmonReport.html'):
        print(f"Previous report file exists. Deleting {fileName}")
        os.remove(fileName)

def getSurveyStats(year):
    dead_to_date_query = f'''
    WITH salmon_counts AS (
        SELECT
            Survey_Date,
            COALESCE(SUM(CASE WHEN Species in ('Chum', 'Coho', 'Unknown', 'Sea-run Cutthroat', 'Sea-run_Cutthroat') AND Type in ('Dead', 'Remnant') THEN Quantity END), 0) AS total_dead_salmon_count,
            COALESCE(SUM(CASE WHEN Species in ('Chum', 'Coho', 'Unknown', 'Sea-run_Cutthroat', 'Sea-run Cutthroat') AND Type = 'Live' THEN Quantity END), 0) AS total_live_salmon_count,
            COALESCE(SUM(CASE WHEN Species in ('Chum', 'Coho', 'Unknown', 'Sea-run_Cutthroat', 'Sea-run Cutthroat') AND Type in ('Live', 'Dead', 'Remnant') THEN Quantity END), 0) AS total_salmon_count,
            COALESCE(SUM(CASE WHEN Species = 'Chum' AND Type in ('Dead', 'Remnant') THEN Quantity END), 0) AS dead_chum_count,
            COALESCE(SUM(CASE WHEN Species = 'Chum' AND Type = 'Live' THEN Quantity END), 0) AS live_chum_count,
            COALESCE(SUM(CASE WHEN Species = 'Coho' AND Type in ('Dead', 'Remnant') THEN Quantity END), 0) AS dead_coho_count,
            COALESCE(SUM(CASE WHEN Species = 'Coho' AND Type = 'Live' THEN Quantity END), 0) AS live_coho_count,
            COALESCE(SUM(CASE WHEN Species in ('Resident_Cutthroat', 'Sea-run_Cutthroat', 'Resident Cutthroat', 'Sea-run Cutthroat', 'Cutthroat') AND Type in ('Dead', 'Remnant') THEN Quantity END), 0) as dead_cutthroat_count,
            COALESCE(SUM(CASE WHEN Species in ('Resident_Cutthroat', 'Sea-run_Cutthroat', 'Resident Cutthroat', 'Sea-run Cutthroat', 'Cutthroat') AND Type = 'Live' THEN Quantity END), 0) as live_cutthroat_count,
            COALESCE(SUM(CASE WHEN Species = 'Unknown' AND Type in ('Dead', 'Remnant') THEN quantity END), 0) AS dead_unknown_count,
            COALESCE(SUM(CASE WHEN Species = 'Unknown' AND Type = 'Live' THEN quantity END), 0) AS live_unknown_count,
            COALESCE(SUM(CASE WHEN Type = 'Redd' THEN Quantity END), 0) as redd_count
        FROM
            salmon
        WHERE
            year = {year} {additionalFilterForAboveDam}
        GROUP BY
            Survey_Date
    ), running_counts AS (
        SELECT
            Survey_Date,
            SUM(dead_chum_count) OVER (ORDER BY Survey_Date) AS running_total_dead_chum,
            SUM(dead_chum_count) OVER (ORDER BY Survey_Date) + live_chum_count AS running_total_all_chum,
            SUM(dead_coho_count) OVER (ORDER BY Survey_Date) AS running_total_dead_coho,
            SUM(dead_coho_count) OVER (ORDER BY Survey_Date) + live_coho_count AS running_total_all_coho,
            SUM(dead_cutthroat_count) OVER (ORDER BY Survey_Date) AS running_total_dead_cutthroat,
            SUM(dead_cutthroat_count) OVER (ORDER BY Survey_Date) + live_cutthroat_count AS running_total_all_cutthroat,
            SUM(dead_unknown_count) OVER (ORDER BY Survey_Date) AS running_total_dead_unknown,
            SUM(dead_unknown_count) OVER (ORDER BY Survey_Date) + live_unknown_count AS running_total_all_unknown,
            SUM(total_dead_salmon_count) OVER (ORDER BY Survey_Date) AS running_total_dead_salmon,
            SUM(total_dead_salmon_count) OVER (ORDER BY Survey_Date) + total_live_salmon_count AS running_total_all_salmon
        FROM
            salmon_counts
    )
    SELECT        
        sc.Survey_Date,
        sc.total_dead_salmon_count,
        sc.total_live_salmon_count,
        sc.total_salmon_count,
        sc.dead_chum_count,
        sc.live_chum_count,
        sc.dead_coho_count,
        sc.live_coho_count,
        sc.dead_cutthroat_count,
        sc.live_cutthroat_count,
        sc.dead_unknown_count,
        sc.live_unknown_count,
        sc.redd_count,
        rc.running_total_dead_chum,
        rc.running_total_all_chum,
        rc.running_total_dead_coho,
        rc.running_total_all_coho,
        rc.running_total_dead_cutthroat,
        rc.running_total_all_cutthroat,
        rc.running_total_dead_unknown,
        rc.running_total_all_unknown,
        rc.running_total_dead_salmon,
        rc.running_total_all_salmon
    FROM
        salmon_counts sc
    JOIN running_counts rc ON sc.Survey_Date = rc.Survey_Date;
    '''
    return pd.read_sql(dead_to_date_query, connection)

def getScatterMap(df, figureTitle):
    fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', color='Type', labels={'Type':'Type'}, color_discrete_map={'Live': 'teal', 'Redd': 'red', 'Dead': 'black'},
                    center=dict(lat=47.71157, lon=-122.3759), zoom=15, hover_name = 'Type', hover_data = ['Distance', 'Quantity', 'Species', 'Sex', 'Accuracy'],
                    mapbox_style='open-street-map', title=figureTitle)
    fig.layout.coloraxis.showscale = False
    fig.update_layout(title_x=0.5)
    fig.show()
    return fig.to_html(include_plotlyjs="cdn")
            
create_salmon_table_query = '''
    CREATE TABLE IF NOT EXISTS salmon (
        _id STRING PRIMARY KEY,
        Survey_Date DATE,
        year DATE,
        Quantity INTEGER,
        Distance INTEGER,
        Stream TEXT,
        Type TEXT,
        Species TEXT,
        Predation TEXT,
        Length FLOAT,
        Width FLOAT,
        Spawned TEXT,
        Sex TEXT,
        Latitude FLOAT,
        Longitude FLOAT,
        Accuracy FLOAT
    );
'''
def getAllFiles():
    !git clone https://github.com/slfisco/Survey-Notebook.git

if IN_COLAB:
    getAllFiles()
else:
    clearPreviousReports()
reddsTable = yearByYearCountPlot = countPlot = surveyStatsTable = yearScatterMap = latestScatterMap = None
maxSurveyChum = maxSurveyChumDate = maxSurveyCoho = maxSurveyCohoDate = chumSpawnSuccess = cohoSpawnSuccess = None   
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()
cursor.execute(create_salmon_table_query)

In [ ]:
### DATA LOADING
import requests

salmon_insert_query = '''
        INSERT OR IGNORE INTO salmon (
        _id,
        Survey_Date,
        year,
        Quantity,
        Distance,
        Stream,
        Type,
        Species,
        Predation,
        Length,
        Width,
        Spawned,
        Sex,
        Latitude,
        Longitude,
        Accuracy
        ) VALUES (?, ?, ?, COALESCE(?,1), ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    '''

def getData(uri):
    response = requests.get(uri)
    return response.json()

## for epicollect data to associate salmon to a survey date
def getSurveyDates(uri):
    surveyDates = {}
    data = getData(uri)
    for entry in data['data']['entries']:
        surveyDate = datetime.strptime(entry['Survey_Date'], "%m/%d/%Y").strftime("%Y-%m-%d")
        surveyDates[entry['ec5_uuid']] = surveyDate
    return surveyDates
    
def getLocation(entry, isEpicollect):
    latitude = longitude = accuracy = location = None
    if isEpicollect:
        latitude = entry.get('Location').get('latitude')
        longitude = entry.get('Location').get('longitude')
        accuracy = entry.get('Location').get('accuracy')
    else:
        location = entry.get('Location')
        if location is not None: 
            location = location.split()
            latitude = location[0]
            longitude = location[1]
            accuracy = location[3]
    return latitude, longitude, accuracy
        
def processEntries(entries, isEpicollect, year, surveyDates):
    for entry in entries:
        location = getLocation(entry, isEpicollect)
        latitude = location[0]
        longitude = location[1]
        accuracy = location[2]
        values = (
            entry.get('ec5_uuid') if isEpicollect else entry.get('_id'),
            surveyDates[entry.get('ec5_parent_uuid')] if isEpicollect else entry.get('Survey_Date'),
            year,
            entry.get('Quantity', 1),
            entry.get('Distance'),
            entry.get('Stream'),
            entry.get('Type'),
            entry.get('Species'),
            entry.get('Predation'),
            entry.get('Length_Inches') if isEpicollect else entry.get("Length"),
            entry.get('Width_Inches') if isEpicollect else entry.get("Width"),
            entry.get('Spawning_Success') if isEpicollect else entry.get("Spawned"),
            entry.get('Sex'),
            latitude,
            longitude,
            accuracy
        )
        cursor.execute(salmon_insert_query, values)
        
def loadSurveyYear(year):
    print(f'loading for year: {year}')
    uri = salmonURIs[year]
    isEpicollect = "epicollect" in uri
    surveyDates = getSurveyDates(surveyURIs[year]) if isEpicollect else None
    allDataInserted = False
    while not allDataInserted:
        data = getData(uri)
        entries = data['data']['entries'] if isEpicollect else data['results']
        processEntries(entries, isEpicollect, year, surveyDates)
        uri = data['links']['next'] if isEpicollect else data['next']
        allDataInserted = True if uri is None else False
        
print('loading salmon into database')        
for year in salmonURIs:
    loadSurveyYear(year)

In [ ]:
def getMaxSurveyTotal(df, columnName):
    max_row = df[columnName].values.argmax()
    total = df.iloc[max_row][columnName]
    return total

def getMaxSurveyDate(df, columnName):
    max_row = df[columnName].values.argmax()
    calcDate = df.iloc[max_row]["Survey_Date"]
    return calcDate
    
df = getSurveyStats(surveyYear)
maxSurveyChum = getMaxSurveyTotal(df, 'running_total_all_chum')
maxSurveyChumDate = getMaxSurveyDate(df, 'running_total_all_chum')
maxSurveyCoho = getMaxSurveyTotal(df, 'running_total_all_coho')
maxSurveyCohoDate = getMaxSurveyDate(df, 'running_total_all_coho')
print(f'max survey chum: {maxSurveyChum}')
print(f'max survey chum date: {maxSurveyChumDate}')
print(f'max survey coho: {maxSurveyCoho}')
print(f'max survey coho date: {maxSurveyCohoDate}')

In [ ]:
def displaySurveyStatsTable():
    tableDf = getSurveyStats(surveyYear)[['Survey_Date', 'live_chum_count', 'dead_chum_count', 'live_coho_count', 'dead_coho_count', 'live_cutthroat_count', 'dead_cutthroat_count', 'live_unknown_count', 'dead_unknown_count']]
    table = tableDf.rename(columns={'Survey_Date': 'Survey Date', 'live_chum_count': 'Live Chum', 'dead_chum_count': 'Dead Chum', 'live_coho_count': 'Live Coho', 'dead_coho_count': 'Dead Coho', 'live_cutthroat_count': 'Live Cutthroat', 'dead_cutthroat_count': 'Dead Cutthroat', 'live_unknown_count': 'Live Unknown', 'dead_unknown_count': 'Dead Unknown'}).style.hide_index().render()
    display(ip.HTML(table))
    return table
surveyStatsTable = displaySurveyStatsTable()

In [ ]:
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
def displayCountPlot(df):
    df['Survey_Date'] = pd.to_datetime(df['Survey_Date'])
    plot = df.plot(ylabel = 'Count', xlabel = 'Survey Date', title = f'{surveyYear} Fish Count', rot=45, xticks=df['Survey_Date'], y=['total_dead_salmon_count', 'total_live_salmon_count', 'live_chum_count', 'dead_chum_count', 'live_coho_count', 'dead_coho_count'], x='Survey_Date')
    plot.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    plot.yaxis.set_major_locator(MaxNLocator(integer=True))
    return getFigureAsHTML()
countPlot = displayCountPlot(getSurveyStats(surveyYear))

In [ ]:
# ### REDDS TABLE. USED TO HELP SURVEY TEAM AVOID REDDS
# redds_table_query = f'''
# SELECT
#     Stream, Distance, Survey_Date
# FROM
#     salmon
# WHERE Type = 'Redd' AND year = {surveyYear}
# '''
# def createReddsTable():
#     table = pd.read_sql(redds_table_query, connection).style.hide_index().render()
#     display(ip.HTML(table))
#     return table
# reddsTable = createReddsTable()

In [ ]:
# ### SPAWN SUCCESS
# def plotSpawnSuccess(species):
#     spawning_query = f'''
#     SELECT
#         CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' AND Spawned = 'Spawned' THEN _id END) AS float) / CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' THEN _id END) AS float) AS spawned_{species}_ratio,
#         CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' AND Spawned = 'Unspawned' THEN _id END) AS float) / CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' THEN _id END) AS float) AS unspawned_{species}_ratio,
#         CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' AND Spawned in ('Partially_spawned', 'Partially spawned') THEN _id END) AS float) / CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' THEN _id END) AS float) AS partial_spawn_{species}_ratio,
#         CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' AND Spawned = 'Unknown' THEN _id END) AS float) / CAST(COUNT(CASE WHEN Species = '{species}' AND Type = 'Dead' THEN _id END) AS float) AS unknown_spawn_{species}_ratio
#     FROM
#         salmon
#     WHERE year = {surveyYear}
#     '''
#     df = pd.read_sql(spawning_query, connection)
#     display(ip.HTML(df.to_html(index=False)))
#     ax = df.plot(kind='barh', stacked=True)
#     return getFigureAsHTML()
# chumSpawnSuccess = plotSpawnSuccess('Chum')
# cohoSpawnSuccess = plotSpawnSuccess('Coho')
# # todo: fix these to same plot

In [ ]:
# ## USER INPUT QUERY
# done = False
# while not done:
#     try:
#         query = input("Enter a query: ")
#         print("entering query: " + query)
#         cursor.execute(query)
#         print(cursor.fetchall())
#     except sqlite3.Error as e:
#         print("SQLite error:", e)

In [ ]:
import unittest
class TestNotebook(unittest.TestCase):
    def testYearlyTotals(self):
        actual = getSurveyStats('2021').tail(1)
        # compare 2021 yearly totals with expected values
        self.assertEqual(actual['running_total_all_salmon'].item(), 1008)
        self.assertEqual(actual['running_total_all_chum'].item(), 939)
        self.assertEqual(actual['running_total_all_coho'].item(), 66)
        self.assertEqual(actual['Survey_Date'].item(), '2021-12-07')
    def testSurveyStats(self):
        # compare 2021-11-16 against expected
        actual = getSurveyStats('2021').query('`Survey_Date` == "2021-11-16"')
        self.assertEqual(actual['dead_chum_count'].item(), 114)
        self.assertEqual(actual['dead_coho_count'].item(), 29)
        self.assertEqual(actual['live_chum_count'].item(), 447)
        self.assertEqual(actual['live_coho_count'].item(), 2)
        self.assertEqual(actual['live_cutthroat_count'].item(), 2)
        self.assertEqual(actual['redd_count'].item(), 39)
        self.assertEqual(actual['total_dead_salmon_count'].item(), 143)
        self.assertEqual(actual['total_live_salmon_count'].item(), 451)
        self.assertEqual(actual['running_total_dead_salmon'].item(), 277)
        self.assertEqual(actual['running_total_dead_chum'].item(), 222)
        self.assertEqual(actual['running_total_dead_coho'].item(), 52)
if not aboveDamOnly: unittest.main(argv=[''], exit=False)

In [ ]:
from datetime import date
def plotSeries(year):
    statsDf = getSurveyStats(year)
    statsDf['Survey_Date'] = statsDf['Survey_Date'].apply(lambda x: datetime.strptime(date.fromisoformat(x).strftime("%m-%d"),"%m-%d"))
    plt.plot('Survey_Date', 'total_salmon_count', data=statsDf, label=year)
def getYearByYearCountPlot():
    fig, ax = plt.subplots()
    for year in salmonURIs:
        plotSeries(year)
    plt.title('Count by time of year')
    plt.ylabel('Count')
    plt.xlabel('Survey Date')
    plt.xticks(rotation = 45)
    plt.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    return getFigureAsHTML()
yearByYearCountPlot = getYearByYearCountPlot()

In [ ]:
## ALL SURVEYS SCATTER MAP
query = f'''
SELECT
    Survey_Date, Type, Species, Latitude, Longitude, Accuracy, Distance, Sex, Quantity
FROM
    salmon
WHERE Latitude IS NOT NULL AND Accuracy < 50 AND year = {surveyYear} {additionalFilterForAboveDam}
'''
df = pd.read_sql(query, connection)
yearScatterMap = getScatterMap(df, f'{surveyYear} Fish Scatter Map')

In [ ]:
##LATEST SURVEY SCATTER MAP
query = f'''
SELECT
    Survey_Date, Type, Species, Latitude, Longitude, Accuracy, Distance, Sex, Quantity
FROM
    salmon
WHERE Latitude IS NOT NULL AND Accuracy < 50 AND year = {surveyYear} AND Survey_Date = (SELECT MAX(Survey_Date) FROM salmon WHERE year = {surveyYear}) {additionalFilterForAboveDam};
'''
df = pd.read_sql(query, connection)
latestSurvey = df.iloc[0]['Survey_Date']
latestScatterMap = getScatterMap(df, f'{latestSurvey} Fish Scatter Map')

In [ ]:
## GENERATE REPORT WITH WHICHEVER FIGURES WERE CREATED
from jinja2 import Environment, FileSystemLoader
def generateReport():
    currentTimePacific = datetime.now(pytz.timezone('America/Los_Angeles')).strftime('%Y-%m-%d_%H-%M-%S') #cannot use system time due to colab
    reportFileName = currentTimePacific + '_salmonReport.html'
    if IN_COLAB:
        templatePath = 'Survey-Notebook/templates'
    else:
        templatePath = 'templates'
    env = Environment(loader=FileSystemLoader(templatePath))
    template = env.get_template('report_template.html')
    reportData = {}
    reportData['reportGenTime'] = currentTimePacific
    reportData['reddsTable'] = reddsTable
    reportData['yearByYearCountPlot'] = yearByYearCountPlot
    reportData['countPlot'] = countPlot
    reportData['surveyStatsTable'] = surveyStatsTable
    reportData['yearScatterMap'] = yearScatterMap
    reportData['latestScatterMap'] = latestScatterMap
    reportData['maxSurveyChum'] = maxSurveyChum
    reportData['maxSurveyChumDate'] = maxSurveyChumDate
    reportData['maxSurveyCoho'] = maxSurveyCoho
    reportData['maxSurveyCohoDate'] = maxSurveyCohoDate
    reportData['chumSpawnSuccess'] = chumSpawnSuccess
    reportData['cohoSpawnSuccess'] = cohoSpawnSuccess
    html = template.render(reportData)
    with open(reportFileName, 'w') as f:
        f.write(html)
generateReport()

In [ ]:
#Download HTML report if in google colab
if IN_COLAB:
    from google.colab import files
    files.download(reportFileName)